# Porosity Wave Simulation - Postprocessing

Load simulation results and analyze energy evolution, deformation fields, and concentration patterns.

In [ ]:
from fenics import *
from utils import *
import numpy as np
import matplotlib.pyplot as plt
import os

# Load parameters - change filename to switch between simulations
pars = read_dictionary('parameter_sets/pars_flowing.json')
#pars = read_dictionary('parameter_sets/pars_diffusive.json')

# Setup
dim, FEu_deg, FE_deg = 2, 2, 1
FEu = VectorElement('CG', triangle, FEu_deg)
FEc = FiniteElement('CG', triangle, FE_deg)
FEmu = FiniteElement('CG', triangle, FE_deg)
FE = MixedElement([FEu, FEc, FEmu])

tau = pars['T'] / pars['n_steps']
k = pars['k']

# Time steps to analyze - modify this list to change which steps are processed
time_indices = [0, 100, 200, 400]

In [ ]:
# Analysis functions
def a(F, c, eta_c, deta_c, pars):
    return 0.5 * pars['mob'] * (c**2) * inner(grad(eta_c), grad(deta_c))

def s(F, c, dot_u, du, pars):
    return 0.5 * pars['visc'] * inner(sym(grad(F*dot_u)), sym(grad(F*du)))

def get_dissipation(q, old_q, tau, pars):
    u, c, eta_c = split(q)
    old_u, old_c, old_eta_c = split(old_q)
    I = Identity(dim)
    old_F = inv(I - grad(old_u))
    dot_u = (u - old_u) / tau
    dot_c = (c - old_c) / tau
    dissi = assemble(s(old_F, old_c, dot_u, dot_u, pars) * dx)
    mobbi = assemble(a(old_F, old_c, eta_c, eta_c, pars) * dx)
    return dissi, mobbi

def energy1(u, c, pars):
    x = SpatialCoordinate(mesh)
    I = Identity(dim)
    F = inv(I - grad(u))
    J = det(F)
    C = F.T*F / J
    E = 0.5*pars['μ']*tr(C - I) 
    Egr = c * inner( Constant((0,pars['g0'])) , x )
    p = (c - pars['c0']) - (J - 1)
    Emix = 0.5*p**2
    Ereg = 0.5*pars['ε']*inner(grad(c),grad(c))
    Etot = E + Egr + Emix + Ereg
    return Etot, E, Egr, Emix, Ereg

def plot_vec(u, nfrac=1.0, **args):
    mesh = u.function_space().mesh()
    x, y, uval, vval = [], [], [], []
    for xy in mesh.coordinates():
        if np.random.rand() <= nfrac:
            val = u(Point(xy))
            x.append(xy[0])
            y.append(xy[1])
            uval.append(val[0])
            vval.append(val[1])
    plt.quiver(x, y, uval, vval, **args)

In [ ]:
# Load simulation data and compute energies
meshes, displacements, Jfields, concentrations, times = [], [], [], [], []
Etot, E, Egr, Emix, Ereg, Diss, Mob = [], [], [], [], [], [], []

for i in time_indices:
    filename = f"{pars['fout']}stateP1_{int(k)}_{pars['n_steps']}_{i}"
    nfilename = f"{pars['fout']}stateP1_{int(k)}_{pars['n_steps']}_{i+1}"
    
    mesh, q, _, t = read_state(filename, FE)
    
    # Dissipation calculation
    mesh1, q1, _, t1 = read_state(nfilename, FE)
    VV = FunctionSpace(mesh1, FE)
    old_q = project(q, VV)
    dissi, mobbi = get_dissipation(q1, old_q, tau, pars)
    Diss.append(dissi)
    Mob.append(mobbi)

    u, c, eta = split(q)
    
    # Deformation quantities
    I = Identity(dim)
    F = I - grad(u)
    J = det(F)
    J_proj = project(J, FunctionSpace(mesh, 'CG', 1))

    # Store data
    times.append(t)
    meshes.append(mesh)
    displacements.append(u)
    Jfields.append(J_proj)
    concentrations.append(c)
    
    # Energy components
    etot, e, egr, emix, ereg = energy1(u, c, pars)
    Etot.append(assemble(etot*dx))
    E.append(assemble(e*dx))
    Egr.append(assemble(egr*dx))
    Emix.append(assemble(emix*dx))
    Ereg.append(assemble(ereg*dx))

print(f"Processed {len(times)} time steps")

In [ ]:
# Energy evolution plot
Etot, E, Egr, Emix, Ereg = np.array(Etot), np.array(E), np.array(Egr), np.array(Emix), np.array(Ereg)
times, Diss, Mob = np.array(times), np.array(Diss), np.array(Mob)

plt.figure(figsize=(12, 8))
plt.plot(times[1:], Etot[1:] - Etot[1], 'k-', linewidth=2, label='Total energy change')
# Uncomment to show individual energy components:
# plt.plot(times[1:], E[1:] - E[1], '--', label='Elastic')
# plt.plot(times[1:], Egr[1:] - Egr[1], '--', label='Gravitational')  
# plt.plot(times[1:], Emix[1:] - Emix[1], '--', label='Mixing')
# plt.plot(times[1:], Ereg[1:] - Ereg[1], '--', label='Regularization')

plt.plot(times[1:], Diss[1:], '--', color='red', label='Viscous dissipation')
plt.plot(times[1:], Mob[1:], '--', color='blue', label='Diffusive dissipation')

plt.legend()
plt.xlabel('Time $t$')
plt.ylabel(r'Energy $\mathcal{H}-\mathcal{H}(q_0)$')
plt.grid(True)
plt.title(f'Energy Evolution - {pars["basename"]}')
plt.savefig(f'energy_{pars["basename"]}.pdf', bbox_inches='tight')
plt.show()

In [ ]:
# Visualization settings - modify these to change plot appearance
cmin, cmax = 0.0, 0.055  # Concentration field range
jmin, jmax = 0.95, 1.01  # Deformation determinant range  
CPRINT = False  # True: plot concentration, False: plot deformation

plt.figure(figsize=(16, 6))
for i, step in enumerate(time_indices[:4]):
    data_idx = list(time_indices).index(step)
    plt.subplot(1, 4, i+1)
    
    mesh = Mesh(meshes[data_idx])
    u_proj = project(displacements[data_idx], VectorFunctionSpace(mesh, 'CG', FEu_deg))
    c_proj = project(concentrations[data_idx], FunctionSpace(mesh, 'CG', 2))
    j_proj = project(Jfields[data_idx], FunctionSpace(mesh, 'CG', 2))
    
    # Optional mesh deformation for visualization:
    # ALE.move(mesh, u_proj)  # Move to deformed config
    # ALE.move(mesh, Constant((1.1*i, 0)))  # Shift horizontally
    
    if CPRINT:
        im = plot(c_proj, vmin=cmin, vmax=cmax, mode='color')
        plt.title(f'Concentration, t={times[data_idx]:.2f}')
    else:
        im = plot(j_proj, mode='color', cmap='jet', vmin=jmin, vmax=jmax)
        plt.title(f'det(F), t={times[data_idx]:.2f}')
    
    plt.colorbar(im, fraction=0.046, pad=0.04)
    # Optional additions:
    # plot_vec(u_proj, nfrac=0.2, scale=1/4*4, scale_units='xy')  # Displacement vectors
    # plot(mesh, linewidth=0.1)  # Mesh lines
    plt.axis('off')

plt.tight_layout()
plt.savefig(f'fields_{pars["basename"]}.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Animation generation (uncomment to create frames)
# Modify frame range and output settings as needed

# plt.figure(figsize=(4, 10))
# frame_range = range(min(100, pars['n_steps']))  # Limit frames
# for i in frame_range:
#     filename = f"{pars['fout']}stateP1_{int(k)}_{pars['n_steps']}_{i}"
#     try:
#         mesh, q, _, t = read_state(filename, FE)
#         u, c, eta = split(q)
#         plt.clf()
#         plot(c, vmin=cmin, vmax=cmax, mode='color')
#         plt.axis('off')
#         plt.title(f'Time: {t:.3f}')
#         plt.savefig(f'ani_{i:04d}.png', dpi=150, bbox_inches='tight')
#     except:
#         break